In [1]:
import pandas as pd

## My API key  & library_Id are stored in another file
api_key,library_id = pd.read_json("Zotero_API_key.json", typ='series')

library_type="group"




from pyzotero import zotero
zot = zotero.Zotero(library_id, library_type, api_key)

ID_Universities='EWSYI3RS' # to access the files in the university subfolder.

In [ ]:
for i in zot.collections_sub(ID_Universities):
    name,key=i["data"]["name"],i["data"]["key"]
    print(f"name: {name} --> key: {key}")

In [ ]:
def relevantItemInfo(dictionary):    
    if dictionary["itemType"]=='attachment':
        # if directory --> key leads to the correct folder
        
        relevant_keys=['key','url']
        newDict={key: dictionary[key] for key in relevant_keys}
        #print(dictionary)
        newDict["note"]=None
       # print(newDict)
        return newDict
    
    if dictionary["itemType"]=="note":
        relevant_keys=['key','note']
        newDict={key: dictionary[key] for key in relevant_keys}
        newDict["url"]=None
        return newDict

    

for i in zot.collection_items(key)[:2]:
    dictionary=i["data"]
    print(dictionary, end="\n\n")
    
    Info=relevantItemInfo(dictionary)
    print(Info, end="\n\n")

In [ ]:
dictList=[]

for i in zot.collections_sub(ID_Universities):
    name,key=i["data"]["name"],i["data"]["key"]
    print(f"name: {name} --> key: {key}")
    
    for i in zot.collection_items(key):
        # only use data if there is an attachment.
        if i["data"]["itemType"]=='attachment':
            Info=relevantItemInfo(i["data"])
            dictList+=[{"Uni": name,"FileKey":Info["key"],"url":Info["url"]}]
    


In [ ]:
df=pd.DataFrame(dictList)
df.head()

In [ ]:
def FileLinks(Filekey):
   #print(Filekey)
    import os
    DIR=f"C:\\Users\\mr\\Zotero\\storage\\{Filekey}"
    #print(DIR)
    FileNames=os.listdir(DIR)
    file=[i for i in FileNames if i.endswith(".html")]
    return DIR+"\\"+FileNames[-1]



df["file"]=df.FileKey.apply(FileLinks)
df.head()

In [ ]:
df.to_json("2023-05-10_Uni_Files_raw.json")

In [ ]:
df=pd.read_json("2023-05-10_Uni_Files_raw.json")

# Add Content from HTMLs

In [ ]:
from htmldate import find_date
#find_date("https://www.uu.nl/en/events/chat-gpt-discover-the-magic-of-ai")

def find_dateX(x):
    try:
        return find_date(x)
    except:
        print(x)


df["date"]=df.url.apply(find_dateX)

In [2]:
import MyLib.nlp as nlp
import MyLib.HTML_prep as HTML_prep
import MyLib.analysis as analysis 
%load_ext autoreload
%autoreload 2

In [ ]:

def FileInfo(FilePath):
    Title,h1,h2,text,linkName,linkUrl=None,None,None,None,None,None
    if FilePath.endswith(".html"):
    
        try:
            #print(FilePath)

            with open(FilePath,"r", encoding='utf-8') as f:
                text= f.read()

            from bs4 import BeautifulSoup

            soup = BeautifulSoup(text, 'html.parser')
            
            Title=soup.title.text
            
            #soup=HTML_prep.removeIMGs(soup,IMG_filename=Title.replace(" ","_"))      


            All_divs=[p.parent for p in soup("p") if p.parent.name=="div"]
            All_divs=(list(set(All_divs)))
            All_divs=[HTML_prep.removeIMGs(div,no_img=True,IMG_filename=Title.replace(" ","_")) for div in All_divs]        

            h1=[item for sublist in [[i.text for i in div("h1")] for div in All_divs if div("h1")!=None] for item in sublist]
            h2=[item for sublist in [[i.text for i in div("h2")] for div in All_divs if div("h2")!=None] for item in sublist]
            text=[item for sublist in [[i.text for i in div("p")] for div in All_divs if div("p")!=None] for item in sublist]
            linkName=[item for sublist in [[i.text for i in div("a")] for div in All_divs if div("a")!=None] for item in sublist]
            linkUrl=[item for sublist in [[i.get('href') for i in div("a")] for div in All_divs if div("a")!=None] for item in sublist]


        except:
            print(f"error with: {FilePath}",end=". ")
    return pd.Series([Title,h1,h2,text,linkName,linkUrl])
        
        

#df.file[2:6].apply(FileInfo)
df[["Title","h1","h2","text","linkName","linkUrl"]]=df.file.apply(FileInfo)


In [ ]:
df.to_json("2023-05-10_Uni_Files_NLP.json")

In [16]:
df=pd.read_json("2023-05-10_Uni_Files_NLP.json")



In [17]:
def Insert_title(text,Title):
    if Title!=None and type(text)==list:
        text=[Title]+text
        return text
    else:
        return text


df["text"]=df[["text","Title"]].apply(lambda x: Insert_title(*x),axis=1)

In [18]:
ChatGPT_Terms="ChatGPT, GPT3, GPT4, OpenAI, AI, LLM, A.I, GPT, Chatbot, technology, tool, app, Artificial Intelligence, Large Language Models"

df["AI_paragraphs"]=df.text.apply(nlp.filter_paragraphs,by=ChatGPT_Terms)



In [53]:
column="AI_paragraphs"

df=df[df.file.apply(lambda x: x.endswith("html"))]

df=df[df.Title!='Research Posters - Faculty of Geosciences - Utrecht University']

df=df.explode(column).reset_index(drop=True)
df=df[df[column].apply(lambda x: type(x)==str)]
df=df.drop_duplicates(column)
df["LEN"]=df[column].apply(lambda x: len(x.split(" ")))


In [54]:
df=nlp.NLP_Pipeline(df, sentiment=False, language="en",translate=True, column=column)

cleaning done.
language detection done.
Translating...
nl Aan de han
nl Sinds de r
nl Tijdens de
nl Jivet wijs
nl In de work
nl Let op: he
nl ChatGPT ke
nl Deze en no
nl Ga het ges
nl Sander Bax
nl ChatGPT sc
nl ChatGPT ka
nl In een wet
nl De komst v
nl Het HOP in
nl ChatGPT ha
nl In het Bri
nl Een van de
nl Eerder mel
nl ChatGPT ka
nl De nieuwe 
nl Ook op Til
nl Maar we zi
nl We vroegen
nl Daarop wer
nl AI gaat im
nl De zes uni
nl “De miljoe
nl Nuijten he
nl Wie al een
nl “We voldoe
nl De TU/e is
nl De start v
nl UM zoekend
nl Nieuwe mog
nl ChatGPT on
nl "Voor de b
nl De geavanc
nl Die heeft 
nl Volgens ha
nl Het is al 
nl En hoe ga 
nl Dat maakt 
nl Universita
nl Google pub
nl Op dinsdag
nl De hype ro
nl Hij is hee
nl Wat ook he
nl In een con
nl "We moeten
nl Kijk eens 
nl Universite
nl Het Univer
nl In haar on
nl In een eer
nl ‘Een prett
nl Ischen ond
nl Onderzoek 
nl Als online
nl Kunstmatig
nl Chatbot ho
nl Kun je ver
nl Sporters w
nl Een voetba
nl Onze mense
nl Docenten b
nl Moet

In [14]:
#df2.text=df2["AI_paragraphs"].apply(lambda x:" ".join(x))

In [ ]:
df2[df2.text.apply(lambda x: type(x))!=str]

In [ ]:
df2["LEN"]=df2.text.apply(lambda x: len(x))
df2=df2[df2.LEN<df2.LEN.max()] ###Cut out the very large list

In [ ]:
df2.LEN.describe()

In [64]:
df=df.reset_index(drop=False)

In [62]:
df.to_json("2023-05-15_Zotero_AI_nlp_en.json")

In [67]:
from ipywidgets import interactive, interact

@interact(column=["NoStopwords"],Mostcommon=(5,200))
def make_wordcloud(column="NoStopwords",Mostcommon=20,Save_as=""):
    hashtags=[i.lower() for s in df[column].dropna() for i in s] # hashtags OR # no stopwords
    analysis.make_wordcloud(hashtags,filename=Save_as,Mostcommon=Mostcommon)

interactive(children=(Dropdown(description='column', options=('NoStopwords',), value='NoStopwords'), IntSlider…

In [ ]:
df3.to_excel("2023-05-08_ZoteroHTMLs.xlsx")